# Simple
One Application with a dependency to one Library

In [ ]:
from cicd_sim import *

In [ ]:
# have reproducible build artifact names
import random
random.seed(123)

## Create Infrastructure

In [ ]:
bitbucket = Repos()
conan = Conan()
artifactory = Artifactory()
jenkins = Jenkins(artifactory, bitbucket)

## Initiate Library

In [ ]:
lib = bitbucket.create_repo('lib')
lib_dev = lib.checkout('develop', 'blue')
lib_dev.commit_file('VERSION', '1.0.0')
lib_dev.push()

## Initiate Application

In [ ]:
app = bitbucket.create_repo('app')
app_dev = app.checkout('develop', 'magenta')
app_dev.commit_file('VERSION', '1.0.0')
app_dev.commit_file('REQUIRES', 'lib/>1.0.0-0')
app_dev.push()

## Create Library 'Feature A'

In [ ]:
feature_a = lib_dev.checkout('feature/A', 'light_gray')
feature_a.commit_file('Feature A File', 'AAAAA')
feature_a.push()

# Merge feature to 'develop'

In [ ]:
lib_dev.merge(feature_a)
lib_dev.push()

# Trigger a manual 'Application' build

In [ ]:
jenkins.build(app_dev)

# Trigger a manual 'Library' build
This will cause yet another application build because the version of the library increased

In [ ]:
jenkins.build(lib_dev)

# Create 'Library release' branch

In [ ]:
lib_r125 = lib_dev.checkout('release/R125', 'yellow')
lib_r125.push()

# Bump library version

In [ ]:
lib_dev.commit_file('VERSION', '1.1.0').push()

# Create 'Application release' branch

In [ ]:
app_release = app_dev.checkout('release/R125', 'yellow').commit_file('REQUIRES', 'lib/>1.0.0-0 <=1.0.0')
app_release.push()
# app_dev.push()
# app_dev.get_file_content('REQUIRES')

# Fix an issue for R125

In [ ]:
lib_r125.commit_file('FixR125', 'A pretty cool fix').push()

# Merge R125 fix back to develop

In [ ]:
lib_dev.merge(lib_r125).push()

# Finalize R125

In [ ]:
lib_master = lib.checkout('master')
lib_master.merge(lib_r125)
lib_r125.delete()
lib_master.push()